# Import packages

In [ ]:
# Standard packages
import numpy as np
import matplotlib.pyplot as plt

# # Parallel processing packages
# import dask
import xarray as xr
# import zarr
# import sarxarray

# # For plotting 
# import contextily as cx
# import pyproj

# # Import required toolboxes
import sys
# sys.path.append('/Users/wangyuqing/software/lambda')
sys.path.append('./')
import LAMBDA

# # For debugging
# import dask
# from dask.distributed import Client
# dask.config.set(scheduler="synchronous") 

In [ ]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.
client

# Set parameters

In [ ]:
# set wavelength of Sentinel-1 (C-band)
wavelength = 0.055465763 #m

# Initial gauss for the sigma of the unknown parameters
sigma_offset = 1 #mm
sigma_vel =  0.5 #mm/yr
sigma_h = 5 #m
sigma_ther = 0.05 #mm/°C

# Load the STM

In [ ]:
stm = xr.open_zarr('./stm_amsterdam_173p.zarr')
# stm = stm.isel(time=range(50))
stm


In [ ]:
# # DEBUG: only use the first 100 time steps
# stm = stm.isel(time=range(100))

In [ ]:
mother_epoch = '20150711'
mother_idx = np.where(stm.time == mother_epoch)[0][0]

dates = stm['dates'].to_numpy()
days = stm['days']
years = stm['years']
# temp = stm.temperature.to_numpy()
temp = stm['temperature']

# Select the point with the smallest NMAD for initialization as the reference point
nmad_init = np.array(stm.nmad_init)
ref_pnt_index= np.argmin(nmad_init)
ref_pnt_index

In [ ]:
# # Make a Spatial plot

# plt.figure(figsize = (15, 10)) 
# ax = plt.subplot(1, 1, 1)
# #fig, ax = plt.subplots(figsize = (15,15))
# ax.scatter(stm.lon.data, stm.lat.data, s=30, color = 'tab:red', label = 'PS')

# # for i in range(0, len(stm.space)-1):
# #     print(i)
# #     pnt_i_idx = i
# #     for j in range(i+1, len(stm.space)):
# for i in range(ref_pnt_index, ref_pnt_index+1):
#     pnt_i_idx = i
#     for j in range(0, len(stm.space)):
#     # for j in range(4, 5):
#         if j != i: 
#             pnt_j_idx = j
#             #ax.scatter(stm.isel(space = pnt_j_idx).lon, stm.isel(space = pnt_j_idx).lat, s=20, color = 'tab:green')
#             ax.plot([stm.isel(space = pnt_i_idx).lon, stm.isel(space = pnt_j_idx).lon], [stm.isel(space = pnt_i_idx).lat, stm.isel(space = pnt_j_idx).lat], color = 'orange', alpha=0.5)
            
# ax.set_xlabel('Longitude', fontsize = 20)
# ax.set_ylabel('Latitude', fontsize = 20)
# ax.legend(loc=4)
# cx.add_basemap(ax, crs='epsg:4326', source=cx.providers.OpenStreetMap.Mapnik, zoom = 17)


# Full batch solution

In [ ]:
# Q_b = np.diag([sigma_offset**2, sigma_vel**2, sigma_h**2, sigma_ther**2])
# A1 = np.diag([-2*np.pi] * len(dates))
# B1 = np.ones((len(dates),4))

# for i in range(ref_pnt_index, ref_pnt_index+1):
#     pnt_i_idx = i
#     for j in range(len(stm.space)):
#         pnt_j_idx = j
#         print(j)


#         # Extract information of the two points of the arc
#         sd_complex_i =  stm.isel(space = pnt_i_idx).sd_complex
#         slc_quality_i = stm.isel(space = pnt_i_idx).slc_quality
#         h2ph_i = stm.isel(space = pnt_i_idx).h2ph_values


#         sd_complex_j=  stm.isel(space = pnt_j_idx).sd_complex
#         slc_quality_j = stm.isel(space = pnt_j_idx).slc_quality
#         h2ph_j = stm.isel(space = pnt_j_idx).h2ph_values


#         # Compute DD for the arc
#         dd_arc = sd_complex_j*sd_complex_i.conj()
#         # # Compute the diagonal of the VCM of the DD phases
#         Qyy_diagonal = np.sqrt((slc_quality_i)**2 + (slc_quality_j)**2)

#         # dd_arc, Qyy_diagonal = compute_dd(sd_complex_i, sd_complex_j, slc_quality_i, slc_quality_j, 0, dates, mother_idx)

#         # Compute the variance covariance matrix of the DD based on the NMAD for the arc
#         Qyy = np.identity(len(stm.time))*Qyy_diagonal.to_numpy()**2

#         # Compute 'mean' h2ph value for the arc (which we currently model as the average of the two time series)
#         h2ph_arc = (h2ph_i + h2ph_j)/2
#         h2ph_arc = h2ph_arc.to_numpy()

#         # Get the wrapped phase
#         phs_wrapped = np.angle(dd_arc)

#         # Define y and the corresponding VQM Qyy
#         y = np.append(phs_wrapped, [0, 0, 0, 0])
#         Q_phs = Qyy
#         # Q_b = np.diag([sigma_offset**2, sigma_vel**2, sigma_h**2, sigma_ther**2])
#         # Qyy = np.block([[Q_phs, np.zeros((Q_phs.shape[0], Q_b.shape[1]))],
#         #                 [np.zeros((Q_b.shape[0], Q_phs.shape[1])), Q_b]])

#         # Define the design matrices
#         # A1 = np.diag([-2*np.pi] * len(dates))
#         # B2 = np.diag([1] * 4)
#         # C = np.block([[A1, np.zeros((A1.shape[0], B2.shape[1]))],
#         #                 [np.zeros((B2.shape[0], A1.shape[1])), B2]])

#         # B1 = np.ones((len(dates),4))
#         B1[:,1] = years*(-4*np.pi/wavelength/1000)
#         B1[:,2] = h2ph_arc*(-4*np.pi/wavelength)
#         B1[:,3] = temp*(-4*np.pi/wavelength/1000)
#         # C[:len(phs_wrapped), -4:] = B1

#         # Float solution with least-squares
#         ahat = phs_wrapped/(-2*np.pi)
#         Qahat = 1/(4*((np.pi)**2)) * (Q_phs + B1@Q_b@B1.T)


#         # Lambda method - Integer bootstrapping
#         afixed,sqnorm,Ps,Qzhat,Z,nfixed,mu = LAMBDA.main(ahat,Qahat,3)

#         # Calculate the unwrapped phase [rad]
#         phs_unw  = phs_wrapped - A1@afixed
        
#         # Get the estimated parameters
#         b_hat = np.linalg.inv((B1.T@np.linalg.inv(Q_phs)@B1))@B1.T@np.linalg.inv(Q_phs)@phs_unw
        
#         # Get the phase for estimated DD observation, non-thermal displacement, height difference and thermal expension [rad]
#         phs_est = B1@b_hat
#         phs_dis =  B1[:,:2]@b_hat[:2]
#         phs_height = B1[:,2]*b_hat[2]
#         phs_ther = B1[:,3]*b_hat[3]

#         # Get the phase for estimated DD observation, non-thermal displacement, height difference and thermal expension [mm]
#         dis_est = phs_est*(-wavelength/(4*np.pi)*1000)
#         dis_dis = phs_dis*(-wavelength/(4*np.pi)*1000)
#         dis_height = phs_height*(-wavelength/(4*np.pi)*1000)
#         dis_ther = phs_ther*(-wavelength/(4*np.pi)*1000)

#         # Get the wrapped and unwrapped phase [mm]
#         dis_wrapped = phs_wrapped*(-wavelength/(4*np.pi)*1000)
#         dis_unw = phs_unw*(-wavelength/(4*np.pi)*1000)


In [ ]:
def full_batch_one_group(stm_pnt, slc_quality_ref, h2ph_ref):
    
    # stm_pnt = stm_pnt.compute()
    slc_quality_pnts = stm_pnt['slc_quality']
    h2ph_pnts = stm_pnt['h2ph_values']
    dd_arc = stm_pnt['dd_complex']
    # # Compute the diagonal of the VCM of the dd phases
    Qyy_diagonal = np.sqrt((slc_quality_ref)**2 + (slc_quality_pnts)**2)


    # Compute the variance covariance matrix of the DD based on the NMAD for the arc
    Qyy = np.identity(len(stm_pnt.time))*Qyy_diagonal.to_numpy()**2

    # Compute 'mean' h2ph value for the arc (which we currently model as the average of the two time series)
    
    h2ph_arc = (h2ph_ref + h2ph_pnts)/2
    # h2ph_arc = h2ph_arc.to_numpy()
    h2ph_arc = h2ph_arc.squeeze().values

    # Get the wrapped phase
    phs_wrapped = np.angle(dd_arc)
    phs_wrapped = phs_wrapped.squeeze()

    # Define y and the corresponding VQM Qyy
    # y = np.append(phs_wrapped, [0, 0, 0, 0])
    Q_phs = Qyy
    Q_b = np.diag([sigma_offset**2, sigma_vel**2, sigma_h**2, sigma_ther**2])
    # Qyy = np.block([[Q_phs, np.zeros((Q_phs.shape[0], Q_b.shape[1]))],
    #                 [np.zeros((Q_b.shape[0], Q_phs.shape[1])), Q_b]])

    # Define the design matrices
    A1 = np.diag([-2*np.pi] * len(phs_wrapped))
    # B2 = np.diag([1] * 4)
    # C = np.block([[A1, np.zeros((A1.shape[0], B2.shape[1]))],
    #                 [np.zeros((B2.shape[0], A1.shape[1])), B2]])

    B1 = np.ones((phs_wrapped.shape[0],4))
    B1[:,1] = years*(-4*np.pi/wavelength/1000)
    B1[:,2] = h2ph_arc*(-4*np.pi/wavelength)
    B1[:,3] = temp*(-4*np.pi/wavelength/1000)
    # C[:len(phs_wrapped), -4:] = B1

    # Float solution with least-squares
    ahat = phs_wrapped/(-2*np.pi)
    Qahat = 1/(4*((np.pi)**2)) * (Q_phs + B1@Q_b@B1.T)


    # Lambda method - Integer bootstrapping
    afixed,sqnorm,Ps,Qzhat,Z,nfixed,mu = LAMBDA.main(ahat,Qahat,3)

    # Calculate the unwrapped phase [rad]
    phs_unw  = phs_wrapped - A1@afixed
    
    # Get the estimated parameters
    b_hat = np.linalg.inv((B1.T@np.linalg.inv(Q_phs)@B1))@B1.T@np.linalg.inv(Q_phs)@phs_unw
    
    # Get the phase for estimated DD observation, non-thermal displacement, height difference and thermal expension [rad]
    phs_est = B1@b_hat
    phs_dis =  B1[:,:2]@b_hat[:2]
    phs_height = B1[:,2]*b_hat[2]
    phs_ther = B1[:,3]*b_hat[3]

    # Get the phase for estimated DD observation, non-thermal displacement, height difference and thermal expension [mm]
    dis_est = phs_est*(-wavelength/(4*np.pi)*1000)
    dis_dis = phs_dis*(-wavelength/(4*np.pi)*1000)
    dis_height = phs_height*(-wavelength/(4*np.pi)*1000)
    dis_ther = phs_ther*(-wavelength/(4*np.pi)*1000)

    # Get the wrapped and unwrapped phase [mm]
    dis_wrapped = phs_wrapped*(-wavelength/(4*np.pi)*1000)
    dis_unw = phs_unw*(-wavelength/(4*np.pi)*1000)

    ds_out = stm_pnt.copy()
    ds_out = ds_out.assign(dis_wrapped=((ds_out.dims), np.expand_dims(dis_wrapped, axis=0)))
    ds_out = ds_out.assign(dis_unw=((ds_out.dims), np.expand_dims(dis_unw, axis=0)))
    ds_out = ds_out.assign(dis_est=((ds_out.dims), np.expand_dims(dis_est, axis=0)))
    ds_out = ds_out.assign(dis_dis=((ds_out.dims), np.expand_dims(dis_dis, axis=0)))
    ds_out = ds_out.assign(dis_height=((ds_out.dims), np.expand_dims(dis_height, axis=0)))
    ds_out = ds_out.assign(dis_ther=((ds_out.dims), np.expand_dims(dis_ther, axis=0)))

    ds_out['vel'] = xr.DataArray(np.array([b_hat[1]]), dims=('space'), coords={'space': ds_out.space.values})
    ds_out['height'] = xr.DataArray(np.array([b_hat[2]]), dims=('space'), coords={'space': ds_out.space.values})
    ds_out['ther'] = xr.DataArray(np.array([b_hat[3]]), dims=('space'), coords={'space': ds_out.space.values})

    return ds_out


In [ ]:
def full_batch_chunk(ds, sd_complex_ref, slc_quality_ref, h2ph_ref):
    """
    Given temporal differnces of point i and point j we compute the double difference phase 
    We also compute the quality of the arc time series 

    point i is the reference point and is subtracted from point j:
    sd_complex_conj_i = sd_complex_i.conj()
    dd_arc = sd_complex_j*sd_complex_conj_i

    Remark that the output of the quality is the sigma of the DD phase. 

    Example:
        dd_arc, Qyy_diagonal_sigma = compute_dd(sd_complex_i, sd_complex_j,sd_quality_i, sd_quality_j, plot, dates, mother_idx)


    Code is as follows:
        sd_complex_conj_i = sd_complex_i.conj()
        dd_arc = sd_complex_j*sd_complex_conj_i
       
        Qyy_diagonal_sigma = np.sqrt((sd_quality_i)**2 + (sd_quality_j)**2)
    """
    ds_out = ds.copy()
    slc_complex_pnts =  ds['sd_complex']
    # slc_quality_pnts = ds['slc_quality']
    sd_complex_conj_ref = sd_complex_ref.conj()
    dd_arc = slc_complex_pnts*sd_complex_conj_ref

    # # # Compute the diagonal of the VCM of the dd phases
    # Qyy_diagonal = np.sqrt((slc_quality_ref)**2 + (slc_quality_pnts)**2)

    ds_out['dd_complex'] = dd_arc
    # ds_out['Qyy_diagonal'] = Qyy_diagonal
    # ds_out['h2ph_arc'] = h2ph_arc

    # # Load chunk in memory
    # ds_out = ds_out.compute()

    groups = ds_out.groupby("space")
    ds_analysis = groups.map(
        full_batch_one_group,
        slc_quality_ref = slc_quality_ref, 
        h2ph_ref = h2ph_ref
    )

    return ds_analysis


In [ ]:
sd_complex_ref_com = stm.isel(space = ref_pnt_index)['sd_complex'].compute()
slc_quality_ref_com = stm.isel(space = ref_pnt_index)['slc_quality'].compute()
h2ph_ref_com = stm.isel(space = ref_pnt_index)['h2ph_values'].compute()

In [ ]:
stm_new = stm.copy()
stm_new['dd_complex'] = stm_new['sd_complex'].copy()
stm_new['Qyy_diagonal'] = stm_new['slc_amplitude'].copy()
stm_new['dis_wrapped'] = stm_new['slc_amplitude'].copy()
stm_new['dis_unw'] = stm_new['slc_amplitude'].copy()
stm_new['dis_est'] = stm_new['slc_amplitude'].copy()
stm_new['dis_dis'] = stm_new['slc_amplitude'].copy()
stm_new['dis_height'] = stm_new['slc_amplitude'].copy()
stm_new['dis_ther'] = stm_new['slc_amplitude'].copy()
stm_new['vel'] = stm_new['nmad_full'].copy()
stm_new['height'] = stm_new['nmad_full'].copy()
stm_new['ther'] = stm_new['nmad_full'].copy()
stm_new

In [ ]:
# stm_new = stm_new.chunk({"space": 20, "time": -1})
# stm_new

In [ ]:
list_keep = ['dd_complex', 'Qyy_diagonal','dis_wrapped', 'dis_unw','dis_est', 'dis_dis', 'dis_height', 'dis_ther','vel', 'height','ther','slc_quality','h2ph_values','sd_complex','slc_amplitude','nmad_full']
list_drop = [var for var in list(stm_new.data_vars.keys()) if var not in list_keep]

In [ ]:
stm_new_in = stm_new.drop_vars(list_drop)
stm_new_in = stm_new_in.chunk({"space": 50, "time": -1})
# stm_new_in = stm_new_in.chunk({"space": 50, "time": 50})

In [ ]:
# dask.visualize(stm_new_in['sd_complex'])

In [ ]:
stm_new2 = xr.map_blocks(full_batch_chunk, stm_new_in, kwargs = {"sd_complex_ref": sd_complex_ref_com, "slc_quality_ref": slc_quality_ref_com, "h2ph_ref": h2ph_ref_com}, template = stm_new_in)
stm_new2

In [ ]:
# dask.visualize(stm_new2['sd_complex'])

In [ ]:
# stm_new2 = stm_new2.chunk({"space": 20, "time": -1})
# stm_new2

In [ ]:
list_keep2 = ['dd_complex', 'Qyy_diagonal','dis_wrapped', 'dis_unw','dis_est', 'dis_dis', 'dis_height', 'dis_ther','vel', 'height','ther']
# list_drop2 = [var for var in list(stm_new2.data_vars.keys()) if var not in list_keep2]

In [ ]:
# stm_new = stm_new.compute()
for var in list_keep2:
    stm_new[var] = stm_new2[var].copy()

In [ ]:
stm_new_comp = stm_new.compute()
# stm_new_comp = stm_new

In [ ]:
stm_new_comp

In [ ]:
# import dask
# dask.visualize(stm_new_comp['Qyy_diagonal'])

In [ ]:
stm_new_comp = stm_new_comp.compute()
# stm_new_comp2 = stm_new_comp2.compute()

In [ ]:
stm_new_comp = stm_new_comp.drop_encoding()
stm_new_comp = stm_new_comp.chunk({"space": 50, "time": -1})
stm_new_comp.to_zarr('test.zarr', mode='w')

In [ ]:
dis_unw0 = stm_new_comp['dis_unw'].isel(space=0).values

In [ ]:
plt.plot(dis_unw0)